In [1]:
import keras.backend

from gernerate_data import load_clas_seg_data
import tensorflow as tf
from sklearn.metrics import classification_report, precision_recall_curve
from keras.utils.np_utils import *
from keras.callbacks import LearningRateScheduler
from sklearn.metrics import classification_report, auc, roc_curve
from models.MTL_IBA import MTL_IBA, MTL_IBA_h3, MTL_IBA_cross, MTL_IBA_cross2, MTL_IBA_cross3
from models.MLT_net import MTL_classic
from sklearn.preprocessing import LabelBinarizer, label_binarize
from utils.losses import dice_coef_loss, dice_coef, dice_coef_loss, focal_tversky, p_r_f1_iou,tversky_loss, binary_crossentropy
from keras.losses import categorical_crossentropy, mean_squared_error, binary_crossentropy
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping, CSVLogger
from keras.optimizers import SGD, Adam
import matplotlib.pyplot as plt
import keras.backend as K
import utils_paths
import numpy as np
import cv2 as cv
import pickle
import os


Using TensorFlow backend.
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:529: FutureWarn


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [ ]:
INIT_LR = 2e-4
EPOCHS = 200
batch_size = 8
depth = 3
img_size = 512
Name = "MTL_IBA_cross3_512"
GPU = True
target = (img_size, img_size)

if GPU:
    os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
    os.environ["CUDA_VISIBLE_DEVICES"] = "1"


print("------------------------------------------------ Reading data ------------------------------------------------")
#
testX_dir = 'dataset/Dataset_BUSI_AN/test/images/'
# model = MTL_classic(img_size, img_size, depth, nClasses=2)
# model = MTL_IBA_cross(img_size, img_size, depth, nClasses=2)
# model = MTL_IBA_cross2(img_size, img_size, depth, nClasses=2)
model = MTL_IBA_cross3(img_size, img_size, depth, nClasses=2)
model.summary()


model.load_weights('MTL_IBA_cross3_512_2e-4.h5', by_name=True)


test_x, test_c_y, test_s_y = load_clas_seg_data(testX_dir, target)


lb = LabelBinarizer()
test_c_y = lb.fit_transform(test_c_y)
test_c_y = to_categorical(test_c_y, 2)


# define callbacks
csv_logger = CSVLogger(Name+'.log')

reduce_lr = ReduceLROnPlateau(monitor='classification_output_loss', factor=0.1, patience=10, min_lr=1e-8, mode='auto', verbose=1)

checkpoint_period1 = ModelCheckpoint(Name + '-{epoch:03d}-{val_segmentation_output_acc:.4f}.h5',
                                     monitor='classification_output_acc', mode='auto', save_best_only='True')

checkpoint_period2 = ModelCheckpoint(Name + '-{epoch:03d}-{val_segmentation_output_acc:.4f}.h5',
                                     monitor='segmentation_output_loss', mode='auto', period=20)


# define loss and compile the model

print("------------------------------------------------ begin training ------------------------------------------------")
opt = Adam(lr=INIT_LR, beta_1=0.9, beta_2=0.99, epsilon=1e-08, decay=0.01)


from keras.losses import categorical_crossentropy

# model.compile(loss={'segmentation_output': tversky_loss, "classification_output": binary_crossentropy},
#               loss_weights={'segmentation_output': 0.3, "classification_output": 0.7},
#               optimizer=opt,
#               metrics={'segmentation_output': ['accuracy', dice_coef], "classification_output": ['accuracy']})

# 
# model.compile(loss={'segmentation_output': categorical_crossentropy, 'seg-out': tversky_loss, "classification_output": binary_crossentropy},
#               loss_weights={'segmentation_output': 0.25,'seg-out': 0.5, "classification_output": 0.5},
#               optimizer=opt, 
#               metrics={'segmentation_output':'accuracy', 'seg-out':dice_coef, 'classification_output':'accuracy'})

model.compile(loss={'segmentation_output': categorical_crossentropy, 'seg-out': tversky_loss, "classification_output": binary_crossentropy},
              loss_weights={'segmentation_output': 0.35,'seg-out': 0.35, "classification_output": 0.3},
              optimizer=opt, 
              metrics={'segmentation_output':'accuracy', 'seg-out':dice_coef, 'classification_output':'accuracy'})

In [ ]:
print("------------------------------------------------class testing ---------------------------------------------------------")
predictions_c, predictions_s, predictions_s2 = model.predict(test_x, batch_size=batch_size)
test_c = test_c_y.argmax(axis=1)
print(classification_report(test_c_y.argmax(axis=1),
      predictions_c.argmax(axis=1),
      digits=6))


# print("------------------------------------------------class testing ---------------------------------------------------------")
# predictions_c, predictions_s = model.predict(test_x, batch_size=batch_size)
# test_c = test_c_y.argmax(axis=1)
# print(classification_report(test_c_y.argmax(axis=1),
#       predictions_c.argmax(axis=1),
#       digits=6))

'''
print("------------------------------------------------ Segmentation testing ------------------------------------------------")

# evaluate the model
# loss = model.evaluate(test_x, [test_c_y, test_s_y], verbose=0)
loss, cla_loss, seg_loss, seg_loss2, cla_acc, seg_acc, seg_dice_coef = model.evaluate(test_x, [test_c_y, test_s_y, test_s_y], verbose=0)
print('Test total loss:', loss)
print('Test classification loss:', cla_loss)
print('Test segmentation loss:', seg_loss)

print('Test classification accuracy:', cla_acc)
print('Test segmentation accuracy:', seg_acc)
print('Test segmentation dice_coef:', seg_dice_coef)


preds_c, preds_s, preds_s2 = model.predict(test_x, batch_size=8, verbose=1)
test_mask = test_s_y.flatten()
pred_mask = preds_s.flatten()
fpr, tpr, thresholds = roc_curve(test_mask, pred_mask, pos_label=1)
roc_auc = auc(fpr, tpr)
print("AUC:", roc_auc)
'''

In [ ]:


import matplotlib.pyplot as plt
lw = 2
plt.figure(figsize=(10, 10))
# ls：折线图的线条风格; lw：折线图的线条宽度; label：标记图内容的标签文本
# 假正率为横坐标，真正率为纵坐标做曲线
plt.plot(fpr, tpr, color='darkorange', lw=lw, label='ROC curve (area = %0.4f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC curve')
# 设置图标
# plt.legend(loc="lower right")
plt.legend(loc="upper right")
plt.show()
plt.savefig('VGG_BUSI-ROC.png', dpi=100, transparent=False)

# Precision, Recall, accuracy, F1, IoU = p_r_f1_iou(test_s_y, preds_s)
# print('Precision:', Precision)
# print('Recall:', Recall)
# print('m_Iou:', IoU)
# print('F1_score:', F1)

## 绘制热力图

In [2]:
from keras.applications.vgg16 import preprocess_input, decode_predictions
from keras.models import load_model, Model
from keras.preprocessing import image
import matplotlib.pyplot as plt
from keras import backend as K
import numpy as np
import cv2



In [3]:
import os
from models.MTL_IBA import MTL_IBA, MTL_IBA_h3, MTL_IBA_cross, MTL_IBA_cross2, MTL_IBA_cross3
from models.MLT_net import MTL_classic
from models.MTL_Attention import MTL_Attention_model

Name =  'MTL_IBA_cross3-2'
img_size =224
depth = 3
testX_dir = 'dataset/Dataset_BUSI_AN/test/images/'
save_dir = 'dataset/prediction/Dataset_BUSI_AN/' + Name
if not os.path.exists(save_dir):
    os.makedirs(save_dir)
    
    

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
   
model = MTL_IBA_cross3(img_size, img_size, depth, nClasses=2)
# model = MTL_Attention_model(img_size, img_size, depth, nClasses=2)
model.load_weights('MTL_IBA_cross3-2.h5', by_name=True)




Instructions for updating:
Colocations handled automatically by placer.


2022-04-03 15:16:50.356927: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2022-04-03 15:16:50.389228: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2200095000 Hz
2022-04-03 15:16:50.393643: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x6528ed0 executing computations on platform Host. Devices:
2022-04-03 15:16:50.393726: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): <undefined>, <undefined>
2022-04-03 15:16:51.094484: I tensorflow/compiler/xla/service/service.cc:150] XLA service 0x65d4f10 executing computations on platform CUDA. Devices:
2022-04-03 15:16:51.094570: I tensorflow/compiler/xla/service/service.cc:158]   StreamExecutor device (0): Tesla V100-PCIE-32GB, Compute Capability 7.0
2022-04-03 15:16:51.095938: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1433] Found device 0 with properties: 
name: Tes

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [4]:
def load_img_preprocess(img_path, target_size):
    img = image.load_img(img_path, target_size=target_size)
    img = image.img_to_array(img) 
    img = np.expand_dims(img, axis=0) # 
    img = preprocess_input(img) #
    return img


def gradient_compute(model, img):
  
    preds = model.predict(img)
    num = np.argmax(preds[0]) #
    
    model2 = Model(inputs=model.input, outputs=model.get_layer('block5_s_conv3').output)

    output = model2.output[0][:, num]
    last_layer = model2.get_layer('block5_s_conv3')
    grads = K.gradients(output, last_layer.output)[0]
    pooled_grads = K.mean(grads, axis=(0, 1, 2)) #
    iterate = K.function([model2.input], [pooled_grads, last_layer.output[0]])
    pooled_grads_value, conv_layer_output_value = iterate([img])

    for i in range(pooled_grads.shape[0]):
        conv_layer_output_value[:, :, i] *= pooled_grads_value[i]

    return conv_layer_output_value

def plot_heatmap(conv_layer_output_value, img_in_path, img_out_path):
    

#     heatmap = np.mean(conv_layer_output_value, axis=-1)
    heatmap = np.mean(conv_layer_output_value,axis = -1)
    heatmap = np.maximum(heatmap, 0)
    heatmap = heatmap / np.max(heatmap)

    img = cv2.imread(img_in_path)
    heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))
    heatmap = np.uint8(255 * heatmap)

    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
    superimopsed_img = heatmap * 0.4 + img
    
    cv2.imwrite(img_out_path, superimopsed_img)
    
    print(img_out_path)

    
def plot_dict_img_heatmap(testX_dir, save_dir, model):
    
    class_list = os.listdir(testX_dir)
    for class_name in class_list:
        class_path = os.path.join(testX_dir, class_name)
        img_list = os.listdir(class_path)
        for image_Name in img_list:
            
            img_path = os.path.join(class_path, image_Name)
            print(img_path)
            img_out_path = os.path.join(save_dir, image_Name)
            
            img = load_img_preprocess(img_path, (224, 224))
            conv_value = gradient_compute(model, img)
            plot_heatmap(conv_value, img_path, img_out_path)
            
#             plot_heatmap(img_path, save_dir, (512, 512) ,model)
 

In [5]:

# model = load_model_h5(model_path)


plot_dict_img_heatmap(testX_dir, save_dir, model)  

dataset/Dataset_BUSI_AN/test/images/bengin_images/benign (51).png


2022-04-03 15:17:28.105921: I tensorflow/stream_executor/dso_loader.cc:152] successfully opened CUDA library libcublas.so.10.0 locally


dataset/prediction/Dataset_BUSI_AN/MTL_IBA_cross3-2/benign (51).png
dataset/Dataset_BUSI_AN/test/images/bengin_images/benign (299).png
dataset/prediction/Dataset_BUSI_AN/MTL_IBA_cross3-2/benign (299).png
dataset/Dataset_BUSI_AN/test/images/bengin_images/benign (167).png
dataset/prediction/Dataset_BUSI_AN/MTL_IBA_cross3-2/benign (167).png
dataset/Dataset_BUSI_AN/test/images/bengin_images/benign (220).png
dataset/prediction/Dataset_BUSI_AN/MTL_IBA_cross3-2/benign (220).png
dataset/Dataset_BUSI_AN/test/images/bengin_images/benign (297).png
dataset/prediction/Dataset_BUSI_AN/MTL_IBA_cross3-2/benign (297).png
dataset/Dataset_BUSI_AN/test/images/bengin_images/benign (425).png
dataset/prediction/Dataset_BUSI_AN/MTL_IBA_cross3-2/benign (425).png
dataset/Dataset_BUSI_AN/test/images/bengin_images/benign (57).png
dataset/prediction/Dataset_BUSI_AN/MTL_IBA_cross3-2/benign (57).png
dataset/Dataset_BUSI_AN/test/images/bengin_images/benign (38).png
dataset/prediction/Dataset_BUSI_AN/MTL_IBA_cross3-2/

dataset/prediction/Dataset_BUSI_AN/MTL_IBA_cross3-2/malignant (106).png
dataset/Dataset_BUSI_AN/test/images/malignant_images/malignant (45).png
dataset/prediction/Dataset_BUSI_AN/MTL_IBA_cross3-2/malignant (45).png
dataset/Dataset_BUSI_AN/test/images/malignant_images/malignant (65).png
dataset/prediction/Dataset_BUSI_AN/MTL_IBA_cross3-2/malignant (65).png
dataset/Dataset_BUSI_AN/test/images/malignant_images/malignant (136).png
dataset/prediction/Dataset_BUSI_AN/MTL_IBA_cross3-2/malignant (136).png
